In [4]:
import numpy as np
from copy import deepcopy
from matplotlib import pyplot as plt

from qiskit import Aer, transpile, assemble, execute
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import IBMQ

token_file = 'api.txt'
with open(token_file, 'r') as file:
    token = file.read().strip()

IBMQ.save_account(token)
provider = IBMQ.load_account()

configrc.store_credentials:WARNING:2023-12-08 16:59:33,336: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2023-12-08 16:59:33,608: Credentials are already in use. The existing account in the session will be replaced.


In [5]:
# The givens: matrix A and vector b
A = np.array([[4, 1, 0, 0],
              [1, 3, 2, 0],
              [0, 2, 3, 1],
              [0, 0, 1, 4]])
b = np.array([1, 2, 2, 1])

# eigenvalues and eigenvectors of matrix A
eigs, vecs = np.linalg.eig(A)


# initializing state vector x_tilde
x_tilde = np.zeros(len(eigs), dtype=complex)

# based on target state vector  we get estimated eigenvalue range 
target_state_index = np.argmax(np.abs(x_tilde))
target_eig_min = np.real(eigs[target_state_index]) - 0.1
target_eig_max = np.real(eigs[target_state_index]) + 0.1


In [6]:
def qft(circ, q, n):
    """Apply the QFT to the first n qubits in the quantum register q."""
    for i in range(n//2):
        circ.swap(q[i], q[n-i-1])
    circ.h(q[0])
    for j in range(n):
        if j+1 < n:
            circ.h(q[j+1])
        for k in range(j):
            circ.cp(np.pi / float(2**(j-k)), q[k], q[j])

def qft_dagger(circ, q, n):
    """Apply the inverse QFT to the first n qubits in the quantum register q."""
    circ.h(q[n-1])
    for j in reversed(range(n)):
        for k in reversed(range(j)):
            circ.cp(-np.pi / float(2**(j-k)), q[k], q[j])
        if j > 0:
            circ.h(q[j-1])
    for i in range(n//2):
        circ.swap(q[i], q[n-i-1])

def qpe_postselection(A, b, eigs, vecs, target_eigval, target_state_index):
  """
  Performs QPE with postselection for a specific eigenvalue and target state.

  Args:
    A: The matrix to solve the eigenvalue problem for.
    b: The vector to find the state vector for.
    eigs: Eigenvalues of A.
    vecs: Eigenvectors of A.
    target_eigval: The desired eigenvalue.
    target_state_index: The index of the target state in the eigenvector.

  Returns:
    A dictionary containing the measured state probabilities for each state.
  """
  nl = QuantumRegister(len(eigs), name='nl')
  nb = QuantumRegister(2, name='nb')
  na = QuantumRegister(1, name='na')
  cb = ClassicalRegister(len(nl), name='cb')
  qc = QuantumCircuit(nb, nl, na, cb)

  t = np.pi  # fixed value for t

  qc.ry(2*np.arccos(b[0]/np.linalg.norm(b)), nb[0])

  for i in range(len(eigs)):
    if eigs[i].real < target_eig_min or eigs[i].real > target_eig_max:
      continue

    qc.h(nl[i])
    qc.p(eigs[i]*t, nl[i])

    qc.cx(nb[0], na[0])
    for j in range(len(vecs[:, i])):
      qc.ry(-2*np.arcsin(abs(vecs[j, i])), na[0])
      if vecs[j, i].imag > 0:
        qc.rz(-np.pi/2, na[0])
      else:
        qc.rz(np.pi/2, na[0])
    qc.cx(nb[0], na[0])

    qft(qc, nl, len(nl))
    qc.measure(nl, cb)

    # Execute the circuit
  backend = Aer.get_backend('qasm_simulator')
  job = execute(qc, backend=backend, shots=12345)
  result = job.result()

  # Analyze results and calculate target state probabilities
  target_state_probabilities = {}
  for key, counts in result.get_counts(qc).items():
    # Check if measured eigenvalue matches desired eigenvalue
    if key[0] != target_eigval:
      continue

    measured_state = qft_dagger(qc, nl, len(nl))
    target_state_probability = abs(measured_state[target_state_index]) ** 2
    target_state_probabilities[key] = (target_state_probability, counts)

  return target_state_probabilities



In [7]:
# he target state index **before** the loop
target_state_index = np.argmax(np.abs(x_tilde))

# QPE with post-selection for each eigenvalue
target_state_probabilities_all = {}

# need post-selection only within the target eigenvalue range
for i, eig in enumerate(eigs):
    if eig.real >= target_eig_min and eig.real <= target_eig_max:
        post_condition = (i, target_state_index)
        target_state_probabilities = qpe_postselection(A, b, eigs, vecs, *post_condition)
        target_state_probabilities_all[eig] = target_state_probabilities

#normalized state vector x_tilde
x_tilde = np.zeros(len(eigs), dtype=complex)

for i, eig in enumerate(eigs):
    tolerance = np.abs(eig) * 0.011  
    # the key will be used  to access the desired eig and stops iterating once a match is found
    for key, (probability, counts) in target_state_probabilities_all[eig].items(): 
        if abs(key[0] - i) < tolerance:
            x_tilde[i] = probability  
            break  

# the state vector normalization 
for i in range(len(x_tilde)):
    if x_tilde[i] == 0:
        x_tilde[i] = 1e-16

x_tilde /= np.linalg.norm(x_tilde)

# Print the complete solution
print(f"x_tilde = {x_tilde}")



KeyError: 5.618033988749891